## This script compares the corrected models with the biomass_C reaction 
(And iYli21 because it also has the `biomass_C` reaction)


Questions:
- Does each model have the biomass reaction 'biomass_C'? - Yes
- Is this reaction for each model the same?
- Does each model have formulas for each metabolite?
- How are the prediction results?

In [3]:
# imports
import cobra
import yaml
from cobra.flux_analysis import pfba
# plotting
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

import sys # append path

sys.path.append('../scripts/')
import helperFunction as hf

# config
config_name = 'model_config'
config_path = f'../config/{config_name}.yaml'

# load config
with open(config_path, 'r') as file:
    config = yaml.safe_load(file)

In [4]:
## experimental data

# # only xu experimental data
yli_glucose_uptake = [0.95, 0.33, 0.72, 0.98, 2.09, 0.61, 0.64, 2.46]
yli_experimental_growth = [0.08, 0.03, 0.07, 0.1, 0.2, 0.047, 0.048, 0.24]


In [5]:
## translate corrected model .mat to .xml

## load corrected models
corr_model_dict = {   # do I need biomass section in the data_dict?
    'yli647': config['models']['yli647'], 
    'iYali4': config['models']['yli4_corr'], 
    'iYli_2.0': config['models']['yli2.0'], 
    'iMK735': config['models']['yliMK735'], 
    'iNL895': config['models']['yliNL895']
}
iYli21_model = cobra.io.read_sbml_model(config['models']['yli21'])

In [53]:
## print the metabolites dict nicely
def metabolites_print(model, rxn_id):
    """Displays the metabolites and their coefficients human readable in the console."""
    rxn = model.reactions.get_by_id(rxn_id)
    print(f'Metabolites for the reaction: {rxn.name} (id: {rxn.id})')
    for metabolite, coefficient in rxn.metabolites.items():
        print(f'{metabolite.name} ({metabolite.id}): {coefficient}')
    return True

#### Analysis of Fructose growth (iMK735 and iYli21)

In [11]:
# read model
iYli2 = cobra.io.read_sbml_model(corr_model_dict['iYli_2.0'])
iMK735 = cobra.io.read_sbml_model(corr_model_dict['iMK735'])
iYali4 = cobra.io.read_sbml_model(corr_model_dict['iYali4'])


No objective in listOfObjectives
No objective coefficients in model. Unclear what should be optimized
No objective in listOfObjectives
No objective coefficients in model. Unclear what should be optimized
No objective in listOfObjectives
No objective coefficients in model. Unclear what should be optimized


In [13]:
# find fructose reaction
with iMK735 as model:
    for rxn in model.exchanges:
        if 'fructose' in rxn.name.lower(): 
            print(rxn.name, rxn.id, rxn.bounds) 
            
# iYli2 D-Fructose exchange R1261 (0.0, 1000.0)
# iYli21 D-fructose exchange R1065 (0.0, 0.0)
# iMK735 D Fructose exchange EX_fru(e) (0.0, 1000.0)

# iYali4 D-fructose exchange 1709 (0.0, 0.0)


D Fructose exchange EX_fru(e) (0.0, 1000.0)


### Each model has the biomass reaction 'biomass_C':

In [54]:
for model_name, model_path in corr_model_dict.items():
    model = cobra.io.read_sbml_model(model_path)
    print(model_name)
    for rxn in model.reactions:
        if 'biomass_C' in rxn.id:
            print(rxn.id, rxn.name)
            print(rxn.reaction)

yli647
biomass_C biomass yarrowia carbon limiting
0.943396927 13BDglcn[c] + 0.567938715 ala_L[c] + 0.055401294 amp[c] + 0.125562993 arg_L[c] + 0.186498253 asn_L[c] + 0.186531036 asp_L[c] + 23.09 atp[c] + 0.868358429 chitin[c] + 0.057210293 cmp[c] + 0.003631558 cys_L[c] + 0.010069587 damp[c] + 0.010436325 dcmp[c] + 0.009226307 dgmp[c] + 0.010353045 dtmp[c] + 0.035250723 ergst[c] + 0.24386559 gln_L[c] + 0.243904342 glu_L[c] + 0.509942819 gly[c] + 0.050870974 gmp[c] + 23.09 h2o[c] + 0.053881088 his_L[c] + 0.090566203 ile_L[c] + 0.194549622 leu_L[c] + 0.275004895 lys_L[c] + 0.235849232 mannan[c] + 0.04423181 met_L[c] + 5.8e-05 pa_SC[c] + 0.000346485 pc_SC[c] + 0.000236765 pe_SC[c] + 0.082570645 phe_L[c] + 0.206371817 pro_L[c] + 3.5e-05 ps_SC[c] + 7.5e-05 ptd1ino_SC[c] + 0.284698809 ser_L[c] + 0.02 so4[c] + 0.210543282 thr_L[c] + 0.006859892 tre[c] + 0.000234034 triglyc_SC[c] + 0.002154449 trp_L[c] + 0.04128234 tyr_L[c] + 0.059363404 ump[c] + 0.17277308 val_L[c] + 0.003029289 zymst[c] --> 2

No objective in listOfObjectives
No objective coefficients in model. Unclear what should be optimized


iYali4
biomass_C biomass_C
0.035251 m1000 + 0.172773 m1008 + 0.943397 m1123 + 0.567939 m114 + 0.243866 m130 + 0.235849 m1324 + 23.09 m141 + 0.059363 m149 + 3.5e-05 m1631 + 0.000234 m1640 + 7.5e-05 m1648 + 0.000237 m1700 + 0.000346 m1701 + 5.8e-05 m1705 + 0.186531 m267 + 0.509943 m272 + 0.00686 m294 + 0.125563 m310 + 0.186498 m319 + 23.09 m32 + 0.003029 m359 + 0.868358 m401 + 0.284699 m441 + 0.003632 m443 + 0.010436 m459 + 0.009226 m465 + 0.243904 m50 + 0.010353 m505 + 0.053881 m615 + 0.044232 m74 + 0.090566 m743 + 0.206372 m765 + 0.210543 m770 + 0.002154 m772 + 0.19455 m775 + 0.275005 m793 + 0.082571 m859 + 0.055401 m86 + 0.01007 m89 + 0.050871 m93 + 0.05721 m95 + 0.02 m964 + 0.041282 m992 --> 23.09 m10 + m1401 + 23.09 m143 + 23.09 m35


No objective in listOfObjectives
No objective coefficients in model. Unclear what should be optimized


iYli_2.0
biomass_C biomass_C
23.09 M00001[c] + 23.09 M00002[c] + 0.055401 M00020[c] + 0.243904 M00025[c] + 0.509943 M00038[c] + 0.567939 M00042[c] + 0.275005 M00048[c] + 0.186531 M00050[c] + 0.05721 M00056[c] + 0.02 M00058[c] + 0.125563 M00061[c] + 0.243866 M00063[c] + 0.284699 M00064[c] + 0.044232 M00072[c] + 0.002154 M00077[c] + 0.082571 M00078[c] + 0.041282 M00081[c] + 0.003632 M00096[c] + 0.059363 M00104[c] + 0.19455 M00122[c] + 0.053881 M00134[c] + 0.050871 M00143[c] + 0.206372 M00146[c] + 0.186498 M00150[c] + 0.000346 M00155[c] + 0.172773 M00180[c] + 0.210543 M00185[c] + 0.010436 M00236[c] + 0.000237 M00342[c] + 0.01007 M00352[c] + 0.009226 M00354[c] + 0.010353 M00356[c] + 0.090566 M00390[c] + 5.8e-05 M00399[c] + 0.000234 M00403[c] + 0.868358 M00434[c] + 0.235849 M00437[c] + 0.943397 M00844[c] + 0.00686 M00927[c] + 7.5e-05 M01014[c] + 0.035251 M01337[c] + 3.5e-05 M01746[c] + 0.003029 M02611[c] --> 23.09 M00008[c] + 23.09 M00009[c] + 23.09 M00079[c] + biomass[e]


No objective in listOfObjectives
No objective coefficients in model. Unclear what should be optimized


iMK735
biomass_C biomass_C
0.943397 13BDglcn[c] + 0.567939 ala_L[c] + 0.055401 amp[c] + 0.125563 arg_L[c] + 0.186498 asn_L[c] + 0.186531 asp_L[c] + 23.09 atp[c] + 0.868358 chitin[c] + 0.05721 cmp[c] + 0.003632 cys_L[c] + 0.01007 damp[c] + 0.010436 dcmp[c] + 0.009226 dgmp[c] + 0.010353 dtmp[c] + 0.035251 ergst[c] + 0.243866 gln_L[c] + 0.243904 glu_L[c] + 0.509943 gly[c] + 0.050871 gmp[c] + 23.09 h2o[c] + 0.053881 his_L[c] + 0.090566 ile_L[c] + 0.19455 leu_L[c] + 0.275005 lys_L[c] + 0.235849 mannan[c] + 0.044232 met_L[c] + 5.8e-05 pa_SC[c] + 0.000346 pc_SC[c] + 0.000237 pe_SC[c] + 0.082571 phe_L[c] + 0.206372 pro_L[c] + 3.5e-05 ps_SC[c] + 7.5e-05 ptd1ino_SC[c] + 0.284699 ser_L[c] + 0.02 so4[c] + 0.210543 thr_L[c] + 0.00686 tre[c] + 0.000234 triglyc_SC[c] + 0.002154 trp_L[c] + 0.041282 tyr_L[c] + 0.059363 ump[c] + 0.172773 val_L[c] + 0.003029 zymst[c] --> 23.09 adp[c] + 23.09 h[c] + 23.09 pi[c]


No objective in listOfObjectives
No objective coefficients in model. Unclear what should be optimized


iNL895
biomass_C biomass_C
0.9434 s_0002 + 7.5e-05 s_0091 + 0.0068599 s_0416 + 0.055401 s_0434 + 23.09 s_0446 + 0.86836 s_0494 + 0.05721 s_0511 + 0.01007 s_0564 + 0.010436 s_0569 + 0.0092263 s_0593 + 0.010353 s_0619 + 0.035251 s_0636 + 0.50994 s_0740 + 0.050871 s_0752 + 0.56794 s_0863 + 0.12556 s_0873 + 0.1865 s_0877 + 0.18653 s_0881 + 0.0036316 s_0889 + 0.2439 s_0899 + 0.24387 s_0907 + 0.053881 s_0911 + 0.090566 s_0920 + 0.19455 s_0925 + 0.275 s_0929 + 0.044232 s_0933 + 0.082571 s_0936 + 0.20637 s_0939 + 0.2847 s_0943 + 0.21054 s_0949 + 0.0021544 s_0952 + 0.041282 s_0955 + 0.17277 s_0960 + 0.23585 s_1011 + 5.8e-05 s_1216 + 3.5e-05 s_1220 + 0.00034649 s_1229 + 0.00023677 s_1234 + 0.02 s_1347 + 0.00023403 s_1399 + 0.059363 s_1417 + 0.0030293 s_1448 --> 23.09 s_0400 + s_0463 + 23.09 s_1207


#### Check for (non-)growth associated maintenance

##### iYli21

In [104]:
# find ATP metabolites and choose cytosolic one -> m141[c] is cytosolic ATP
# find all reactions and choose the maintenance
for met in iYli21_model.metabolites:
    if 'm141[c]' == met.id:
        for rxn in met.reactions: # => xMAINTENANCE 
            if 'xMAINTENANCE' == rxn.id:
                print(rxn.id, ';', rxn.name, rxn.reaction, rxn.bounds)
                # => m141[c] --> m143[c] + m35[c] (7.8625, 1000.0)

maintenance = iYli21_model.reactions.get_by_id('xMAINTENANCE')
hf.reactionInformationPrint([maintenance])
hf.formulaWithNames(maintenance)

xMAINTENANCE ; Non-growth associated maintenance (NGAM) m141[c] --> m143[c] + m35[c] (7.8625, 1000.0)
Non-growth associated maintenance (NGAM), (id: xMAINTENANCE) is active. (Bounds: (7.8625, 1000.0))


'ATP_C10H16N5O13P3 --> ADP_C10H15N5O10P2 + phosphate_H3O4P'

##### Using ATP metabolite


In [105]:
for model_name, model_path in corr_model_dict.items():
    model = cobra.io.read_sbml_model(model_path)
    print(model_name)

    for met in model.metabolites:
        if 'ATP' in met.name:
            for rxn in met.reactions:
                if 'main' in rxn.id or 'main' in rxn.name:
                    print(met.id, met.name)
                    print(rxn.id, ';', rxn.name)
                    print(rxn.reaction)
                    print(rxn.bounds)

        if 'iYli_2.0' == model_name: # used to find ATP maintenance reaction: found "ATP phosphohydrolase" id: R0542
            if 'M00002[c]' == met.id:
                for rxn in met.reactions:
                    for reactant in rxn.reactants:
                        if 'ATP' in reactant.name:
                            for product in rxn.products:
                                if 'ADP' in product.name:
                                    print(rxn.id, ';', rxn.name)
        
        # if 'iNL895' == model_name: # found nothing
        #     if 's_0446' == met.id: # s_0446 ATP[cy] 
        #         for rxn in met.reactions:
        #             print(rxn.id, ';', rxn.name)

# iYli21: Non-growth associated maintenance (NGAM), (id: xMAINTENANCE): m141[c] --> m143[c] + m35[c] (Bounds: (7.8625, 1000.0))
# yli647: ATP maintenance: id: ATPM: atp[c] + h2o[c] --> adp[c] + h[c] + pi[c] bounds: (0.0, 1000.0)
# iYali4: ATP maintenance: id: xMAINTENANCE: m141 --> m143 + m35 bounds: (0.0, 1000.0)
# iYli_2.0: ATP phosphohydrolase: id: R0542: (Bounds: (7.8625, 1000.0))
# iMK735: ATP maintenance: id: ATPM: atp[c] + h2o[c] --> adp[c] + h[c] + pi[c] bounds: (1.0, 1.0)
# iNL895: nothing found



yli647
atp[c] ATP
ATPM ; ATP maintenance requirement
atp[c] + h2o[c] --> adp[c] + h[c] + pi[c]
(0.0, 1000.0)


No objective in listOfObjectives
No objective coefficients in model. Unclear what should be optimized


iYali4
m141 ATP[cy]
xMAINTENANCE ; Non-growth associated maintenance (NGAM)
m141 --> m143 + m35
(0.0, 1000.0)


No objective in listOfObjectives
No objective coefficients in model. Unclear what should be optimized


iYli_2.0
R0048 ; Urea:carbon-dioxide ligase (ADP-forming)
R0304 ; ATP:glycerone phosphotransferase
R0352 ; 1-phosphatidylinositol 3-kinase, yeast-specific
R0570 ; ATP:shikimate 3-phosphotransferase
R0295 ; gamma-L-glutamyl-L-cysteine:glycine ligase (ADP-forming)
R0042 ; ATP:L-glutamate 5-phosphotransferase
R0305 ; ATP:glycerol 3-phosphotransferase
R0033 ; ATP:D-glucosamine 6-phosphotransferase
R0542 ; ATP phosphohydrolase
R0615 ; ATP:cytidine 5'-phosphotransferase
R0806 ; ATP:4-methyl-5-(2-hydroxyethyl)-thiazole 2-phosphotransferase
R1323 ; ATP:N-acetyl-D-glucosamine 6-phosphotransferase
R0369 ; ATP:3-phospho-D-glycerate 1-phosphotransferase
R0422 ; L-2-Aminoadipate-6-semialdehyde:NADP+ 6-oxidoreductase
R0805 ; ATP:4-amino-5-hydroxymethyl-2-methylpyrimidine 5-phosphotransferase
R0457 ; 5-Formyltetrahydrofolate cyclo-ligase (ADP-forming)
R1088 ; ADP/ATP transporter, mitochondrial
R0379 ; ATP:D-fructose-6-phosphate 1-phosphotransferase
R0351 ; phosphatidylinositol 4-kinase, yeast-specifi

No objective in listOfObjectives
No objective coefficients in model. Unclear what should be optimized


iMK735
atp[c] ATP
ATPM ; ATP maintenance requirement
atp[c] + h2o[c] --> adp[c] + h[c] + pi[c]
(1.0, 1.0)


No objective in listOfObjectives
No objective coefficients in model. Unclear what should be optimized


iNL895


In [95]:

# ### load iYli_2.0
# ID_OF_INTEREST = 'R0448' # NO NAME
# ID_OF_INTEREST = 'R2000' # R2000 name and id
# ID_OF_INTEREST = 'R0542' # ATP phosphohydrolase (Bounds: (7.8625, 1000.0))
ID_OF_INTEREST = 'R0436' # ATP phosphohydrolase (Bounds: (0.0, 1000.0))
iYli_20 = cobra.io.read_sbml_model(corr_model_dict['iYli_2.0'])
rxnasdf = iYli_20.reactions.get_by_id(ID_OF_INTEREST)
hf.reactionInformationPrint([rxnasdf])
hf.formulaWithNames(rxnasdf)

# # load iNL895
ID_OF_INTEREST = 'r_0244' # ATP diphosphohydrolase (Bounds: (0.0, 1000.0))
iNL895 = cobra.io.read_sbml_model(corr_model_dict['iNL895'])
rxnasdf = iNL895.reactions.get_by_id(ID_OF_INTEREST)
hf.reactionInformationPrint([rxnasdf])
hf.formulaWithNames(rxnasdf)

No objective in listOfObjectives
No objective coefficients in model. Unclear what should be optimized


ATP phosphohydrolase, (id: R0436) is active. (Bounds: (0.0, 1000.0))


No objective in listOfObjectives
No objective coefficients in model. Unclear what should be optimized


ATP diphosphohydrolase, (id: r_0244) is active. (Bounds: (0.0, 1000.0))


'ATP[cy] --> AMP[cy] + 2.0 phosphate[cy]'

In [48]:
# iYali4 = cobra.io.read_sbml_model(corr_model_dict['iYali4'])
# # Find Non-growth associated maintenance
# for model_name, model_path in corr_model_dict.items():
#     print(model_name)
#     model = cobra.io.read_sbml_model(model_path)
#     for rxn in model.reactions:
#         if 'mainten' in rxn.name:
#             print(rxn.id, rxn.name)
#             print(rxn.reaction)

# xMAINTENANCE = iYali4.reactions.get_by_id('xMAINTENANCE')
# xMAINTENANCE

# yli647
# ATPM ATP maintenance requirement
# atp[c] + h2o[c] --> adp[c] + h[c] + pi[c]
# iYali4
# xMAINTENANCE Non-growth associated maintenance (NGAM)
# m141 --> m143 + m35
# iYli_2.0 => no maintenance reaction
# iMK735
# ATPM ATP maintenance requirement
# atp[c] + h2o[c] --> adp[c] + h[c] + pi[c]
# iNL895 => no maintenance reaction
# yli647 = cobra.io.read_sbml_model(corr_model_dict['yli647'])
# atp_maintenance = yli647.reactions.get_by_id('ATPM')
# atp_maintenance # bounds: (0,1000)
# iYali4 = cobra.io.read_sbml_model(corr_model_dict['iYali4'])
# xMAINTENANCE = iYali4.reactions.get_by_id('xMAINTENANCE')
# xMAINTENANCE # bounds: (0,1000)
# iYli_2_0 = cobra.io.read_sbml_model(corr_model_dict['iYli_2.0']) # no maintenance reaction
# iMK735 = cobra.io.read_sbml_model(corr_model_dict['iMK735'])
# atp_maintenance = iMK735.reactions.get_by_id('ATPM')
# atp_maintenance # bounds: (1,1)
# iNL895 = cobra.io.read_sbml_model(corr_model_dict['iNL895']) # no maintenance reaction



No objective in listOfObjectives
No objective coefficients in model. Unclear what should be optimized


Reaction identifier,xMAINTENANCE
Name,Non-growth associated maintenance (NGAM)
Memory address,0x2a75e01c0
Stoichiometry,m141 --> m143 + m35 ATP[cy] --> ADP[cy] + phosphate[cy]
GPR,
Lower bound,0.0
Upper bound,1000.0


### Informatinos for the meeting on 19.04


In [49]:
### biomass_C reaction (for maintenance)
iYali4 = cobra.io.read_sbml_model(corr_model_dict['iYali4'])

# biomass
biomass_C = iYali4.reactions.get_by_id('biomass_C')
biomass_C

# xMAINTENANCE = iYali4.reactions.get_by_id('xMAINTENANCE')


No objective in listOfObjectives
No objective coefficients in model. Unclear what should be optimized


Reaction identifier,biomass_C
Name,biomass_C
Memory address,0x2ab241850
Stoichiometry,0.035251 m1000 + 0.172773 m1008 + 0.943397 m1123 + 0.567939 m114 + 0.243866 m130 + 0.235849 m1324 + 23.09 m141 + 0.059363 m149 + 3.5e-05 m1631 + 0.000234 m1640 + 7.5e-05 m1648 + 0.000237 m1700 +... 0.035251 ergosterol[cy] + 0.172773 L-valine[cy] + 0.943397 (1-3)-beta-D-glucan[cy] + 0.567939 L-alanine[cy] + 0.243866 L-glutamine[cy] + 0.235849 mannan[cy] + 23.09 ATP[cy] + 0.059363 UMP[cy] +...
GPR,
Lower bound,0.0
Upper bound,1000.0


### Is the biomass_C reaction the same for each model?

In [40]:
##one example
# example_name = 'iYali4'
# model = cobra.io.read_sbml_model(corr_model_dict[example_name])
# print(example_name)

# biomass_rxn_id = 'biomass_C'
# biomass_rxn = model.reactions.get_by_id(biomass_rxn_id)
# metabolites_print(model, biomass_rxn_id)

# example_name = 'yli647'
# model_path = corr_model_dict[example_name]
# model = cobra.io.read_sbml_model(model_path)
# print(example_name)

# biomass_rxn_id = 'biomass_C'
# biomass_rxn = model.reactions.get_by_id(biomass_rxn_id)
# metabolites_print(model, biomass_rxn_id)

############################ compare names and coefficients for all models
# biomass_rxn_id = 'biomass_C'
# for model_name, model_path in corr_model_dict.items():
#     model = cobra.io.read_sbml_model(model_path)
#     print(model_name)
#     metabolites_print(model, biomass_rxn_id)

# print('iYli21')
# metabolites_print(iYli21_model, biomass_rxn_id)


yli647
Metabolites for the reaction: biomass yarrowia carbon limiting (id: biomass_C)
1_3_beta_D_Glucan (13BDglcn[c]): -0.943396927
H2O (h2o[c]): -23.09
L_Aspartate (asp_L[c]): -0.186531036
ATP (atp[c]): -23.09
AMP (amp[c]): -0.055401294
Trehalose (tre[c]): -0.006859892
L_Glutamate (glu_L[c]): -0.243904342
L_Arginine (arg_L[c]): -0.125562993
L_Glutamine (gln_L[c]): -0.24386559
L_Alanine (ala_L[c]): -0.567938715
Glycine (gly[c]): -0.509942819
L_Methionine (met_L[c]): -0.04423181
L_Proline (pro_L[c]): -0.206371817
L_Asparagine (asn_L[c]): -0.186498253
CMP (cmp[c]): -0.057210293
L_Cysteine (cys_L[c]): -0.003631558
dAMP (damp[c]): -0.010069587
dCMP (dcmp[c]): -0.010436325
dGMP (dgmp[c]): -0.009226307
dTMP (dtmp[c]): -0.010353045
Ergosterol (ergst[c]): -0.035250723
Chitin__monomer_ (chitin[c]): -0.868358429
GMP (gmp[c]): -0.050870974
L_Histidine (his_L[c]): -0.053881088
L_Isoleucine (ile_L[c]): -0.090566203
L_Leucine (leu_L[c]): -0.194549622
L_Lysine (lys_L[c]): -0.275004895
Mannan (mannan[

No objective in listOfObjectives
No objective coefficients in model. Unclear what should be optimized


iYali4
Metabolites for the reaction: biomass_C (id: biomass_C)
H2O[cy] (m32): -23.09
L-glutamate[cy] (m50): -0.243904
L-methionine[cy] (m74): -0.044232
AMP[cy] (m86): -0.055401
dAMP[cy] (m89): -0.01007
GMP[cy] (m93): -0.050871
CMP[cy] (m95): -0.05721
L-alanine[cy] (m114): -0.567939
L-glutamine[cy] (m130): -0.243866
ATP[cy] (m141): -23.09
UMP[cy] (m149): -0.059363
L-aspartate[cy] (m267): -0.186531
L-glycine[cy] (m272): -0.509943
trehalose[cy] (m294): -0.00686
L-arginine[cy] (m310): -0.125563
L-asparagine[cy] (m319): -0.186498
zymosterol[cy] (m359): -0.003029
chitin_H2O(C8H13NO5)n[cy] (m401): -0.868358
L-serine[cy] (m441): -0.284699
L-cysteine[cy] (m443): -0.003632
dCMP[cy] (m459): -0.010436
dGMP[cy] (m465): -0.009226
dTMP[cy] (m505): -0.010353
L-histidine[cy] (m615): -0.053881
L-isoleucine[cy] (m743): -0.090566
L-proline[cy] (m765): -0.206372
L-threonine[cy] (m770): -0.210543
L-tryptophan[cy] (m772): -0.002154
L-leucine[cy] (m775): -0.19455
L-lysine[cy] (m793): -0.275005
L-phenylalanine

No objective in listOfObjectives
No objective coefficients in model. Unclear what should be optimized


iYli_2.0
Metabolites for the reaction: biomass_C (id: biomass_C)
L-Glutamine[c] (M00063[c]): -0.243866
L-Glutamate[c] (M00025[c]): -0.243904
H2O[c] (M00001[c]): -23.09
ATP[c] (M00002[c]): -23.09
L-Alanine[c] (M00042[c]): -0.567939
L-Aspartate[c] (M00050[c]): -0.186531
Glycine[c] (M00038[c]): -0.509943
AMP[c] (M00020[c]): -0.055401
L-Asparagine[c] (M00150[c]): -0.186498
Chitin[c] (M00434[c]): -0.868358
L-Arginine[c] (M00061[c]): -0.125563
L-Proline[c] (M00146[c]): -0.206372
L-Methionine[c] (M00072[c]): -0.044232
L-Cysteine[c] (M00096[c]): -0.003632
L-Serine[c] (M00064[c]): -0.284699
L-Threonine[c] (M00185[c]): -0.210543
Phosphatidate[c] (M00399[c]): -5.8e-05
Triacylglycerol[c] (M00403[c]): -0.000234
Phosphatidylcholine[c] (M00155[c]): -0.000346
Phosphatidylethanolamine[c] (M00342[c]): -0.000237
CMP[c] (M00056[c]): -0.05721
Phosphatidylserine[c] (M01746[c]): -3.5e-05
1-Phosphatidyl-D-myo-inositol[c] (M01014[c]): -7.5e-05
L-Histidine[c] (M00134[c]): -0.053881
L-Lysine[c] (M00048[c]): -0.2

No objective in listOfObjectives
No objective coefficients in model. Unclear what should be optimized


iMK735
Metabolites for the reaction: biomass_C (id: biomass_C)
1_3_beta_D_Glucan (13BDglcn[c]): -0.943397
L_Alanine (ala_L[c]): -0.567939
AMP (amp[c]): -0.055401
L_Arginine (arg_L[c]): -0.125563
L_Asparagine (asn_L[c]): -0.186498
L_Aspartate (asp_L[c]): -0.186531
ATP (atp[c]): -23.09
Chitin__monomer_ (chitin[c]): -0.868358
CMP (cmp[c]): -0.05721
L_Cysteine (cys_L[c]): -0.003632
dAMP (damp[c]): -0.01007
dCMP (dcmp[c]): -0.010436
dGMP (dgmp[c]): -0.009226
dTMP (dtmp[c]): -0.010353
Ergosterol (ergst[c]): -0.035251
L_Glutamine (gln_L[c]): -0.243866
L_Glutamate (glu_L[c]): -0.243904
Glycine (gly[c]): -0.509943
GMP (gmp[c]): -0.050871
H2O (h2o[c]): -23.09
L_Histidine (his_L[c]): -0.053881
L_Isoleucine (ile_L[c]): -0.090566
L_Leucine (leu_L[c]): -0.19455
L_Lysine (lys_L[c]): -0.275005
Mannan (mannan[c]): -0.235849
L_Methionine (met_L[c]): -0.044232
Phosphatidate__yeast_specific (pa_SC[c]): -5.8e-05
Phosphatidylcholine__yeast_specific (pc_SC[c]): -0.000346
phosphatidylethanolamine__yeast_speci

No objective in listOfObjectives
No objective coefficients in model. Unclear what should be optimized


iNL895
Metabolites for the reaction: biomass_C (id: biomass_C)
(1->3)-beta-D-glucan[cy] (s_0002): -0.9434
1-phosphatidyl-1D-myo-inositol[cy] (s_0091): -7.5e-05
alpha,alpha-trehalose[cy] (s_0416): -0.0068599
AMP[cy] (s_0434): -0.055401
ATP[cy] (s_0446): -23.09
chitin[cy] (s_0494): -0.86836
CMP[cy] (s_0511): -0.05721
dAMP[cy] (s_0564): -0.01007
dCMP[cy] (s_0569): -0.010436
dGMP[cy] (s_0593): -0.0092263
dTMP[cy] (s_0619): -0.010353
ergosterol[cy] (s_0636): -0.035251
L-glycine[cy] (s_0740): -0.50994
GMP[cy] (s_0752): -0.050871
L-alanine[cy] (s_0863): -0.56794
L-arginine[cy] (s_0873): -0.12556
L-asparagine[cy] (s_0877): -0.1865
L-aspartate[cy] (s_0881): -0.18653
L-cysteine[cy] (s_0889): -0.0036316
L-glutamate[cy] (s_0899): -0.2439
L-glutamine[cy] (s_0907): -0.24387
L-histidine[cy] (s_0911): -0.053881
L-isoleucine[cy] (s_0920): -0.090566
L-leucine[cy] (s_0925): -0.19455
L-lysine[cy] (s_0929): -0.275
L-methionine[cy] (s_0933): -0.044232
L-phenylalanine[cy] (s_0936): -0.082571
L-proline[cy] (s

True

#### Weird findings (iMK735 has no biomass metabolite, iNL895 does not have H2O and H):

In [69]:
# iMK735: biomass_C does not include the biomass metabolite
# looking for the biomass metabolite in this model
# 1. load model
# iMK735 = cobra.io.read_sbml_model(corr_model_dict['iMK735'])

# 2. find biomass metabolite => not found 
# for met in iMK735.metabolites:
#     if 'biomass' in met.name.lower():
#         print(met.name, met.id)

# # 3. check all biomass reactions => there is nothing as a biomass reaction
# for rxn in iMK735.exchanges:
#     if 'biomass' in rxn.name.lower():
#         print(rxn.id, rxn.name)
#         print(rxn.reaction)

### Look into iNL895 (does it have H2O and H as metabolites?)
# load model
# iNL895_model = cobra.io.read_sbml_model(corr_model_dict['iNL895'])

# # No metabolite names H2O or H
# for met in iNL895_model.metabolites:
#     if 'H' in met.name:
#         print(met.name, met.id)

# look into NAD -> NADH definition s_1087: NADH[cy], s_1082: NAD[cy]
# nadh = iNL895_model.metabolites.get_by_id('s_1087')
# for rxn in nadh.reactions:
#     print(rxn.metabolites)
#     for met in rxn.metabolites:
#         if 'nad' in met.name.lower():
#             print(met.name, met.id, rxn.id, rxn.reaction)

## find NADH producing reaction
# r_0742 = iNL895_model.reactions.get_by_id('r_0742')
# r_0742 # NADH[cy] + ubiquinone-6[mi] --> NAD[cy] + ubiquinol-6[mi]

# r_0734 = iNL895_model.reactions.get_by_id('r_0734')
# r_0734 # 4-acetamidobutanal[cy] + NAD[cy] --> 4-acetamidobutanoate[cy] + NADH[cy]

# # set biomass and look into summary
# iNL895_model.objective = 'biomass_C'
# iNL895_model.metabolites.get_by_id('s_1087').summary()

# # => found producing reactions: r_0525, r_0688, r_020_xxx, r_0064, r_0969, r_0575
# r_0525 = iNL895_model.reactions.get_by_id('r_0525')
# r_0525 # glyceraldehyde 3-phosphate[cy] + NAD[cy] + phosphate[cy] <=> 1,3-bisphospho-D-glycerate[cy] + NADH[cy]

# r_0688 = iNL895_model.reactions.get_by_id('r_0688')
# r_0688 # (S)-malate[cy] + NAD[cy] <=> NADH[cy] + oxaloacetate(2-)[cy]

# r_020_xxx = iNL895_model.reactions.get_by_id('r_020_xxx')
# r_020_xxx # IMP[cy] + NAD[cy] --> xanthosine-5-phosphate[cy] + NADH[cy]

# r_0064 = iNL895_model.reactions.get_by_id('r_0064')
# r_0064 # (2R,3S)-3-isopropylmalate[cy] + NAD[cy] --> (2S)-2-isopropyl-3-oxosuccinate(2-)[cy] + NADH[cy]

# r_0969 = iNL895_model.reactions.get_by_id('r_0969')
# r_0969 # L-saccharopine[cy] + NAD[cy] <=> 2-oxoglutarate[cy] + L-lysine[cy] + NADH[cy]

# r_0575 = iNL895_model.reactions.get_by_id('r_0575')
# r_0575 # L-histidinol[cy] + 2.0 NAD[cy] --> L-histidine[cy] + 2.0 NADH[cy]


Reaction identifier,r_0525
Name,glyceraldehyde-3-phosphate dehydrogenase
Memory address,0x7fe42daa45e0
Stoichiometry,"s_0731 + s_1082 + s_1207 <=> s_0265 + s_1087 glyceraldehyde 3-phosphate[cy] + NAD[cy] + phosphate[cy] <=> 1,3-bisphospho-D-glycerate[cy] + NADH[cy]"
GPR,YALI0C06369g
Lower bound,-1000.0
Upper bound,1000.0


### Does all corrected models have chemical formulas?

In [84]:
# check number of metabolites with chemical formula and without and print it nicely
for model_name, model_path in corr_model_dict.items():
    model = cobra.io.read_sbml_model(model_path)
    model.objective = 'biomass_C'
    print(model_name)
    print('Number of metabolites: ', len(model.metabolites))
    ## look if chemical formulas exist
    metabolite_counter = 0
    for met in model.metabolites:
        if met.formula is not None:
            metabolite_counter += 1
            continue
    print(f'Number of metabolites in {model_name} with a chemical formula is {metabolite_counter}/{len(model.metabolites)} = {round(metabolite_counter/len(model.metabolites),3)}% (=> {len(model.metabolites) - metabolite_counter} whithout) a chemical formula')

## iNL895 has 0 metabolites with a chemical formula?
# iNL895_model = cobra.io.read_sbml_model(corr_model_dict['iNL895'])

# for met in iNL895_model.metabolites:
#     print(met.id, met.name, met.formula)


yli647
Number of metabolites:  1121
Number of metabolites in yli647 with a chemical formula is 1101/1121 = 0.982% (=> 20 whithout) a chemical formula


No objective in listOfObjectives
No objective coefficients in model. Unclear what should be optimized


iYali4
Number of metabolites:  1683
Number of metabolites in iYali4 with a chemical formula is 1277/1683 = 0.759% (=> 406 whithout) a chemical formula


No objective in listOfObjectives
No objective coefficients in model. Unclear what should be optimized


iYli_2.0
Number of metabolites:  1087
Number of metabolites in iYli_2.0 with a chemical formula is 988/1087 = 0.909% (=> 99 whithout) a chemical formula


No objective in listOfObjectives
No objective coefficients in model. Unclear what should be optimized


iMK735
Number of metabolites:  1115
Number of metabolites in iMK735 with a chemical formula is 1076/1115 = 0.965% (=> 39 whithout) a chemical formula


No objective in listOfObjectives
No objective coefficients in model. Unclear what should be optimized


iNL895
Number of metabolites:  1590
Number of metabolites in iNL895 with a chemical formula is 0/1590 = 0.0% (=> 1590 whithout) a chemical formula


### Check the molecular weigth of biomass with the summary function

In [89]:
##one example
example_name = 'iYali4'
model = cobra.io.read_sbml_model(corr_model_dict[example_name])
model.objective = 'xBIOMASS'
print(example_name)

No objective in listOfObjectives
No objective coefficients in model. Unclear what should be optimized


iYali4


In [93]:
## Check the molecular weight of the biomass reaction 
### Accoring to this source: https://stackoverflow.com/questions/43796431/calculating-molecular-weight-in-python
### 1. function for formula parsing H20 to [('H', 2), ('O', 1)], (parsed_formula = parse_formula(formula))
### 2. weight dict (weight = {'C' : 12.0110, ...})
### 3. calculate weight: return sum([weight[symbol] * number] for symbol, number in parsed_formula])

#### 1 + 2 + 3: solved by: https://github.com/cgohlke/molmass#examples

### BUT: There are 1. metabolites without formula and metabolites whicht consist of multiple metabolites (e.g. proteins, lipids)

# 1. Iteriere über die metabolites in der biomass reaction 
# a.) liste von einfachen -> python molmass
# b.) liste von komplizierten (e.g. chitin, proteins, lipids)


# 3. Multipliziere das Gewicht mit dem Wert in der biomass reaction und addiere es auf


m32 H2O[cy] H2O
m86 AMP[cy] C10H14N5O7P
m89 dAMP[cy] C10H14N5O6P
m93 GMP[cy] C10H14N5O8P
m95 CMP[cy] C9H14N3O8P
m141 ATP[cy] C10H16N5O13P3
m149 UMP[cy] C9H13N2O9P
m294 trehalose[cy] C12H22O11
m401 chitin_H2O(C8H13NO5)n[cy] None
m459 dCMP[cy] C9H14N3O7P
m465 dGMP[cy] C10H14N5O7P
m505 dTMP[cy] C10H15N2O8P
m1123 (1-3)-beta-D-glucan[cy] None
m1324 mannan[cy] None
m1726 protein[cy] None
m1727 lipids[cy] None


In [98]:
#### find chitin number of elements (using summary)
model.metabolites.get_by_id('m401').summary()
# chitin = model.metabolites.get_by_id('chitin_H2O(C8H13NO5)n[cy]')

# # compare to glucose reaction summary
# model.metabolites.get_by_id('m511').summary() # -> 10mmol/h

# example for chitin: 0.3556 mmol/h (wikipedia: formula: C8H13NO5, weight: 203.19 g/mol ~ 0.20319 g/mmol) => 0.3556 * 0.20316 = 0.072 g/h

Percent,Flux,Reaction,Definition
100.00%,0.3556,272,m403 --> m10 + m11 + m401
Percent,Flux,Reaction,Definition
100.00%,-0.3556,xBIOMASS,0.4415 m1123 + 0.1104 m1324 + 86.7881 m141 + 0.0623 m149 + m1726 + m1727 + 0.0032 m294 + 86.7881 m32 + 0.4068 m401 + 0.0377 m459 + 0.0377 m465 + 0.0383 m505 + 0.0757 m86 + 0.0383 m89 + 0.093 m93 + 0.0578 m95 --> 86.7881 m10 + m1401 + 86.7881 m143 + 86.7881 m35


In [90]:
## Fast way to compute MW (hopefully) 
# => 1. Find biomass metabolite 2. Use summary 3. Get MW from summary

## 1. Find biomass metabolite (If existend)
for met in model.metabolites:
    if 'biomass' in met.name.lower():
        print(met.name, met.id)
biomass_metabolite = 'm1401'

# 2. Use summary
model.metabolites.get_by_id(biomass_metabolite).summary()


biomass[cy] m1401


Percent,Flux,Reaction,Definition
100.00%,0.8741,xBIOMASS,0.4415 m1123 + 0.1104 m1324 + 86.7881 m141 + 0.0623 m149 + m1726 + m1727 + 0.0032 m294 + 86.7881 m32 + 0.4068 m401 + 0.0377 m459 + 0.0377 m465 + 0.0383 m505 + 0.0757 m86 + 0.0383 m89 + 0.093 m93 + 0.0578 m95 --> 86.7881 m10 + m1401 + 86.7881 m143 + 86.7881 m35
Percent,Flux,Reaction,Definition
100.00%,-0.8741,2111,m1401 -->


### Investigate growth reaction or alternatives to biomass_C (maybe from iYLI647)

In [8]:
#### helper function
# find biomass or growth reactions:
def find_biomass_or_growth_reactions(model, print_reactions = True):
    biomass_or_growth_reactions = []
    for rxn in model.reactions:
        if 'biomass' in rxn.name.lower() or 'growth' in rxn.name.lower():
            if print_reactions:
                print(rxn.name, rxn.id)
            biomass_or_growth_reactions.append(rxn)
    return biomass_or_growth_reactions

#### iYLI647

In [ ]:
# load model
yli647 = cobra.io.read_sbml_model(corr_model_dict['yli647'])

In [9]:
with yli647 as model:
    find_biomass_or_growth_reactions(model)


biomass yarrowia carbon limiting biomass_C
biomass yarrowia nitrogen limiting biomass_N


#### iYali4

In [11]:
# load model
iYali4 = cobra.io.read_sbml_model(corr_model_dict['iYali4'])

No objective in listOfObjectives
No objective coefficients in model. Unclear what should be optimized


In [12]:
with iYali4 as model:
    find_biomass_or_growth_reactions(model)

yeast 5 biomass pseudoreaction 2110
growth 2111
yeast 6 biomass pseudoreaction 2133
yeast 8 biomass pseudoreaction 4041
Non-growth associated maintenance (NGAM) xMAINTENANCE
xBIOMASS xBIOMASS
biomass_C biomass_C


In [34]:
## investigate xBIOMASS
metabolites_print(iYali4, 'xBIOMASS') # ATP (id: m141) -86.7881

Metabolites for the reaction: xBIOMASS (id: xBIOMASS)
H2O[cy] (m32): -86.7881
AMP[cy] (m86): -0.0757
dAMP[cy] (m89): -0.0383
GMP[cy] (m93): -0.093
CMP[cy] (m95): -0.0578
ATP[cy] (m141): -86.7881
UMP[cy] (m149): -0.0623
trehalose[cy] (m294): -0.0032
chitin_H2O(C8H13NO5)n[cy] (m401): -0.4068
dCMP[cy] (m459): -0.0377
dGMP[cy] (m465): -0.0377
dTMP[cy] (m505): -0.0383
(1-3)-beta-D-glucan[cy] (m1123): -0.4415
mannan[cy] (m1324): -0.1104
protein[cy] (m1726): -1.0
lipids[cy] (m1727): -1.0
H+_p+1[cy] (m10): 86.7881
phosphate[cy] (m35): 86.7881
ADP[cy] (m143): 86.7881
biomass[cy] (m1401): 1.0


True

In [28]:
# check results for simulation on 2.43 mmol glucose gDW/h 0.64 and 0.61 respectively
# ## get glucose reaction => D-glucose exchange (id): 1714
# for rxn in iYali4.reactions:
#     if 'glucose' in rxn.name.lower():
#         print(rxn.name, rxn.id)

# find experimental data
yli_glucose_uptake = [2.43, 0.64] # from Guo et al. 2020
yli_experimental_growth = [0.26, 0.048]

# set objective: 
iYali4.objective = 'xBIOMASS'

# set glucose uptake
glucose_exchange_reaction = '1714'
biomass_reaction = 'xBIOMASS'

# set conditions + simulate
iYali4_simulation_growth = []
for condition in yli_glucose_uptake:
    with iYali4 as model:
        dGlu = model.reactions.get_by_id(glucose_exchange_reaction)
        dGlu.bounds = (-condition, 1000)
        try:
            pfba_solution = pfba(model)
            growthRate = pfba_solution.fluxes[biomass_reaction]
        except:
            growthRate = 0
        iYali4_simulation_growth.append(growthRate)
        print(f'Growth rate on {condition} mmol/h glucose: {growthRate}')

# results from Guo et al. 2020 are not done with objective either

Growth rate on 2.43 mmol/h glucose: 0.21241122007295252
Growth rate on 0.64 mmol/h glucose: 0.05594369582168275


In [14]:
## investigate growth reaction (id: 2111)
iYali4.reactions.get_by_id('2111')

Reaction identifier,2111
Name,growth
Memory address,0x29fa47fa0
Stoichiometry,m1401 --> biomass[cy] -->
GPR,
Lower bound,0.0
Upper bound,1000.0


#### iMK735

In [15]:
# load model
iMK735 = cobra.io.read_sbml_model(corr_model_dict['iMK735'])

No objective in listOfObjectives
No objective coefficients in model. Unclear what should be optimized


In [16]:
with iMK735 as model:
    find_biomass_or_growth_reactions(model)

biomass yarrowia 0.4% bal YL biomass_004
biomass yarrowia 1.3% bal YL biomass_013
biomass yarrowia 5.1% bal YL biomass_051
biomass yarrowia 20% bal YL biomass_20
biomass yarrowia 40% bal YL biomass_40
biomass yarrowia 60% bal YL biomass_60
biomass yarrowia 80% bal YL biomass_80
biomass WOtag YL biomass_WOtag
biomass WOtag and glycogen YL biomass_WOtG
biomass_C biomass_C


In [18]:
## investigate biomass_013
iMK735.reactions.get_by_id('biomass_013').metabolites # => atp: -17.05

{<Metabolite 13BDglcn[c] at 0x2a61c0370>: -1.16963,
 <Metabolite ala_L[c] at 0x2a61df6a0>: -0.47277,
 <Metabolite amp[c] at 0x2a61dfa30>: -0.0474,
 <Metabolite arg_L[c] at 0x2a61dfd30>: -0.16559,
 <Metabolite asn_L[c] at 0x2a61dfe50>: -0.1048,
 <Metabolite asp_L[c] at 0x2a61dff40>: -0.30656,
 <Metabolite atp[c] at 0x2a61df160>: -17.05,
 <Metabolite cmp[c] at 0x2a61e1940>: -0.04606,
 <Metabolite cys_L[c] at 0x2a61e1dc0>: -0.0068,
 <Metabolite damp[c] at 0x2a61e1040>: -0.00371,
 <Metabolite dcmp[c] at 0x2a61ea310>: -0.00247,
 <Metabolite dgmp[c] at 0x2a61ea730>: -0.00247,
 <Metabolite dtmp[c] at 0x2a61ead90>: -0.00371,
 <Metabolite ergst[c] at 0x2a61f4130>: -0.00072,
 <Metabolite gln_L[c] at 0x2a61f4d60>: -0.10861,
 <Metabolite glu_L[c] at 0x2a61f4ee0>: -0.31099,
 <Metabolite gly[c] at 0x2a61ff130>: -0.29924,
 <Metabolite glycogen[c] at 0x2a61ff2b0>: -0.53429,
 <Metabolite gmp[c] at 0x2a61ff310>: -0.0474,
 <Metabolite h2o[c] at 0x2a61ff670>: -17.05,
 <Metabolite his_L[c] at 0x2a61ffeb0>: